In [ ]:
%matplotlib inline 
%matplotlib nbagg
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import tensorflow as tf
from tensorflow.python.framework.ops import reset_default_graph

import data
import utils
import custom_ops

In [ ]:
from tensorflow.contrib.layers import fully_connected, batch_norm

reset_default_graph()

# Defining model
num_iterations = 3e4
batch_size=64
number_inputs=42
number_outputs=8
seq_len=30 # max 700
learning_rate = 0.001

X_input = tf.placeholder(tf.float32, shape=[None, None, number_inputs], name='X_input')
X_length = tf.placeholder(tf.int32, shape=[None,], name='X_length')
t_input = tf.placeholder(tf.int32, shape=[None, None], name='t_input')
t_mask = tf.placeholder(tf.float32, shape=[None, None], name='t_mask')

num_units_encoder = 100
num_units_l1 = 100

cell_fw = tf.nn.rnn_cell.GRUCell(num_units_encoder)
cell_bw = tf.nn.rnn_cell.GRUCell(num_units_encoder)
#enc_cell = tf.nn.rnn_cell.OutputProjectionWrapper(enc_cell, number_outputs)
enc_outputs, _ = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell_fw, cell_bw=cell_bw, inputs=X_input,
                                                 sequence_length=X_length, dtype=tf.float32)
enc_outputs = tf.concat(2, enc_outputs)
outputs = tf.reshape(enc_outputs, [-1, num_units_encoder*2])
l1 = fully_connected(outputs, num_units_l1, normalizer_fn=batch_norm)
l_out = fully_connected(l1, number_outputs, activation_fn=None)

batch_size_shp = tf.shape(enc_outputs)[0]
seq_len_shp = tf.shape(enc_outputs)[1]
l_out_reshape = tf.reshape(l_out, [batch_size_shp, seq_len_shp, number_outputs])

y = l_out_reshape

In [ ]:
def loss_and_acc(preds):
    # sequence_loss_tensor is a modification of TensorFlow's own sequence_to_sequence_loss
    # TensorFlow's seq2seq loss works with a 2D list instead of a 3D tensors
    loss = custom_ops.sequence_loss(preds, t_input, t_mask)
    # if you want regularization
    #reg_scale = 0.00001
    #regularize = tf.contrib.layers.l2_regularizer(reg_scale)
    #params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
    #reg_term = sum([regularize(param) for param in params])
    #loss += reg_term
    # calculate accuracy
    argmax = tf.to_int32(tf.argmax(preds, 2))
    correct = tf.to_float(tf.equal(argmax, t_input)) * t_mask
    accuracy = tf.reduce_sum(correct) / tf.reduce_sum(t_mask)
    return loss, accuracy, argmax

loss, accuracy, predictions = loss_and_acc(y)

# use lobal step to keep track of our iterations
global_step = tf.Variable(0, name='global_step', trainable=False)
# pick optimizer, try momentum or adadelta
optimizer = tf.train.AdamOptimizer(learning_rate)
# extract gradients for each variable
grads_and_vars = optimizer.compute_gradients(loss)
# add below for clipping by norm
#gradients, variables = zip(*grads_and_vars)  # unzip list of tuples
#clipped_gradients, global_norm = (
#    tf.clip_by_global_norm(gradients, self.clip_norm) )
#grads_and_vars = zip(clipped_gradients, variables)
# apply gradients and make trainable function
train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

In [ ]:
X_train, X_valid, t_train, t_valid, mask_train, mask_valid, length_train, length_valid, num_seq_train =\
data.get_train(seq_len)
print("X_train,", X_train.shape, X_train.dtype)
print("t_train,", t_train.shape, t_train.dtype)
print("mask_train,", mask_train.shape, mask_train.dtype)
print("length_train,", length_train.shape, length_train.dtype)
print("num_seq_train", num_seq_train)
print("X_valid,", X_valid.shape, X_valid.dtype)
print("t_valid,", t_valid.shape, t_valid.dtype)
print("mask_valid,", mask_valid.shape, mask_valid.dtype)
print("length_valid,", length_valid.shape, length_valid.dtype)

In [ ]:
# restricting memory usage, TensorFlow is greedy and will use all memory otherwise
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)
# initialize the Session
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts))
# test train part
sess.run(tf.initialize_all_variables())
feed_dict = {X_input: X_valid, X_length: length_valid, t_input: t_valid,
             t_mask: mask_valid}
fetches = [y]
res = sess.run(fetches=fetches, feed_dict=feed_dict)
print "y", res[0].shape

In [ ]:
def shuffle_data(X, t, mask, length):
    num_sequences = X.shape[0]
    seq_names = np.arange(0,num_sequences)
    np.random.shuffle(seq_names)
    X = X_train[seq_names]
    t = t[seq_names]
    mask = mask[seq_names]
    length = length[seq_names]
    return X, t, mask, length

In [ ]:
# setting up running parameters
val_interval = batch_size*2
samples_to_process = 1e5
samples_processed = 0
num_samples_per_epoch = int(num_seq_train/batch_size)
samples_val = []
costs, accs_val = [], []
plt.figure()
try:
    while samples_processed < samples_to_process:
        #if (num_samples_per_epoch % samples) == 0:
        #    print("shuffling data ...")
        #    X_train, t_train, mask_train, length_train = shuffle_data(
        #        X_train, t_train, mask_train, length_train)
        # make fetches
        fetches_tr = [train_op, loss, accuracy]
        # set up feed dict
        feed_dict_tr = {X_input: X_train, X_length: length_train,
                        t_input: t_train, t_mask: mask_train}
        # run the model
        res = tuple(sess.run(fetches=fetches_tr, feed_dict=feed_dict_tr))
        _, batch_cost, batch_acc = res
        costs += [batch_cost]
        samples_processed += batch_size
        #print("samples_processed,", samples_processed)
        #print("batch_cost,", batch_cost)
        #if samples_processed % 1000 == 0: print batch_cost, batch_acc
        #validation data
        if samples_processed % val_interval == 0:
            print "validating"
            fetches_val = [accuracy, y]
            feed_dict_val = {X_input: X_valid, X_length: length_valid, t_input: t_valid,
                             t_mask: mask_valid}
            res = tuple(sess.run(fetches=fetches_val, feed_dict=feed_dict_val))
            acc_val, output_val = res
            samples_val += [samples_processed]
            accs_val += [acc_val]
            #print("validation_accs", acc_val)
            plt.plot(samples_val, accs_val, 'g-')
            plt.ylabel('Validation Accuracy', fontsize=15)
            plt.xlabel('Processed samples', fontsize=15)
            plt.title('', fontsize=20)
            plt.grid('on')
            plt.savefig("out.png")
            display.display(display.Image(filename="out.png"))
            display.clear_output(wait=True)
except KeyboardInterrupt:
    pass